# AFCluster -> MSA Transformer Analysis

### Setup AFCluster

In [ ]:
#@title Setup AFCluster
%%bash
if [ ! -d AF_cluster ];then
  git clone https://github.com/AkanoLoki/AF_cluster.git
fi
if [ ! -d output ]; then
  mkdir output
fi
pip -q install ml-collections dm-haiku biopython polyleven

In [ ]:
#@title Cluster MSA
keyword = 'test' #@param {type:"string"}
#@markdown - Specify the output identifier string
a3mInput = '/content/test.a3m' #@param {type:"string"}
#@markdown - Specify the input a3m file
outputDir = '/content/subsampled_MSAs' #@param {type:"string"}
#@markdown - Specify the output directory of clustered MSAs (will mkdir if folder not present)
msaTDir = '/content/MSATransformer_output' #@param {type:"string"}
#@markdown - Specify the output directory of MSA Transformer (will mkdir if folder not present)
oligoState = 1 #@param {type:"integer"}
#@markdown - Specify the oligomeric state of the protein
if oligoState < 1:
    raise Exception("Oligomeric state cannot be smaller than 1")
import os
os.system(f"python /content/AF_cluster/scripts/ClusterMSA.py {keyword} -i {a3mInput} -o {outputDir} --n_mer {oligoState}")

### Setup ESM

In [ ]:
#@title Setup ESM dependencies
%pip -q install biotite
%pip -q install git+https://github.com/facebookresearch/esm.git
!apt-get install aria2
!mkdir -p /root/.cache/torch/hub/checkpoints
!aria2c --dir=/root/.cache/torch/hub/checkpoints --continue --split 8 --max-connection-per-server 8 https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50S.pt
!aria2c --dir=/root/.cache/torch/hub/checkpoints --continue --split 8 --max-connection-per-server 8 https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50S-contact-regression.pt
!aria2c --dir=/root/.cache/torch/hub/checkpoints --continue --split 8 --max-connection-per-server 8 https://dl.fbaipublicfiles.com/fair-esm/models/esm_msa1b_t12_100M_UR50S.pt
!aria2c --dir=/root/.cache/torch/hub/checkpoints --continue --split 8 --max-connection-per-server 8 https://dl.fbaipublicfiles.com/fair-esm/regression/esm_msa1b_t12_100M_UR50S-contact-regression.pt

In [ ]:
#@title Invoke HWS ESM MSA-T Script
os.system(f"python /content/AF_cluster/scripts/runESM.py {outputDir}/*a3m -o {msaTDir} --model msa_t")

### excerpted from data_sep2022/00_KaiB/KaiB_landscape_figures.ipynb

In [ ]:
#@title Setup plot dependencies
%pylab inline
%matplotlib inline
import numpy as np
import pandas as pd

import seaborn as sns
sns.set_style('ticks')
sns.set_context('paper')

import os
from scipy.stats import ttest_ind

blue = sns.color_palette('Blues')[-3]
red =  sns.color_palette('Reds')[-3]
grey= sns.color_palette('Greys')[-3]

def jitterbox(**kwargs):
    'supply x, y, hue, data'
    sns.stripplot(**kwargs, dodge=True, alpha=0.5,zorder=0)
    ax = sns.boxplot(**kwargs, dodge=True, fliersize=0, zorder=10, boxprops = dict(facecolor=(0,0,0,0)))

    handles, labels = ax.get_legend_handles_labels()
    l = plt.legend(handles[0:3], labels[0:3], bbox_to_anchor=(1, 1), frameon=False)

In [ ]:
df = pd.read_json('KaiB_feats_3recycles.json.zip')
df['cluster_ind'] = [os.path.basename(x.replace('.pdb','')).split('_')[-1] for x in df['pdb']]

def get_type(pdb):
    end = pdb.split('_')[-1]
    if end=='REF.pdb':
        return 'REF'
    elif end.startswith('U'):
        return end.split('-')[0]
    else:
        return 'Tree'
    
df['Type'] = df.apply(lambda row: get_type(row['pdb']), axis=1)

In [ ]:
def get_msaT(cluster_ind):
    return np.loadtxt('msaT_preds/msa_t__2QKEE_%s.npy' % cluster_ind)

tree_df = df.loc[df.Type=='Tree']
tree_df['msaT_arr'] = tree_df.apply(lambda row: get_msaT(row['cluster_ind']), axis=1)

In [ ]:
figure(figsize=(9,3))

df = df.sort_values('mean_pLDDT')
maxx=1.75
subplot(1,3,1)

tmp = df.loc[df.Type=='Tree']
scatter(tmp['rmsd_ref_2QKEE'], tmp['rmsd_ref_5JYTA'], c=tmp['mean_pLDDT'], cmap='rainbow_r',vmin=50,vmax=90)

#colorbar(aspect=3.5,label='MSA size',fraction=0.05,orientation='horizontal',ticks=[1e1,1e2,1e3])
xlabel('RMSD to Ground state (2QKE)')
ylabel('RMSD to Fold-switch state (5JYT)')

xlim([0,maxx])
ylim([0,maxx])
title('Clustered sampling')

subplot(1,3,2)

tmp = df.loc[df.Type=='U10']
scatter(tmp['rmsd_ref_2QKEE'], tmp['rmsd_ref_5JYTA'], c=tmp['mean_pLDDT'], cmap='rainbow_r',vmin=50,vmax=90)

title('Uniform sampling, |MSA| = 10')
#colorbar(aspect=3.5,label='MSA size',fraction=0.05,orientation='horizontal',ticks=[1e1,1e2,1e3])
xlabel('RMSD to Ground state (2QKE)')
ylabel('RMSD to Fold-switch state (5JYT)')

xlim([0,maxx])
ylim([0,maxx])
#

subplot(1,3,3)

tmp = df.loc[df.Type=='U100']
scatter(tmp['rmsd_ref_2QKEE'], tmp['rmsd_ref_5JYTA'], c=tmp['mean_pLDDT'], cmap='rainbow_r',vmin=50,vmax=90)
title('Uniform sampling, |MSA| = 100')

#colorbar(aspect=3.5,label='MSA size',fraction=0.05,orientation='horizontal',ticks=[1e1,1e2,1e3])
xlabel('RMSD to Ground state (2QKE)')
ylabel('RMSD to Fold-switch state (5JYT)')

xlim([0,maxx])
ylim([0,maxx])
tight_layout()

#savefig('kaib_landscape.pdf',bbox_inches='tight')

In [ ]:
 # get representative clusters of first and second state
state_1 = df.loc[df['rmsd_ref_5JYTA']<0.3] #[df['mean_pLDDT']>87.5]
print(len(state_1))

state_2 = df.loc[df['rmsd_ref_2QKEE']<0.3] #[df['mean_pLDDT']>87.5]
print(len(state_2))

bkgd = df.loc[df['rmsd_ref_2QKEE']>=0.3][df['rmsd_ref_5JYTA']>=0.3]
state_1['state'] = 'FS state'
state_2['state'] = 'Ground state'
bkgd['state'] = 'Other'

df_w_states = pd.concat([state_1, state_2, bkgd])

figure(figsize=(3,3))

jitterbox(x='Type', y='mean_pLDDT', hue='state', data=df_w_states, 
              order=['U100','U10','Tree'],
              hue_order=['Other', 'FS state', 'Ground state'], palette=[grey, red, blue])
ylabel('plDDT')
ylim([30,100])
xticks(range(3), ['Uniform\n|MSA|=100', 'Uniform\n|MSA|=10', 'Clustered'])
xlabel('')
legend([],frameon=False)

tight_layout()

#savefig('kaib_state_plddts.pdf',bbox_inches='tight')


In [ ]:
#@title Setup ESM python imports
from typing import List, Tuple, Optional, Dict, NamedTuple, Union, Callable
import itertools
import os
import string
from pathlib import Path

import numpy as np
import torch
from scipy.spatial.distance import squareform, pdist, cdist
import matplotlib.pyplot as plt
import matplotlib as mpl
from Bio import SeqIO
import biotite.structure as bs
from biotite.structure.io.pdbx import PDBxFile, get_structure
from biotite.database import rcsb
from tqdm import tqdm
import pandas as pd

import esm

torch.set_grad_enabled(False)

In [6]:
#@title Setup ESM functions
# This is an efficient way to delete lowercase characters and insertion characters from a string
deletekeys = dict.fromkeys(string.ascii_lowercase)
deletekeys["."] = None
deletekeys["*"] = None
translation = str.maketrans(deletekeys)

def read_sequence(filename: str) -> Tuple[str, str]:
    """ Reads the first (reference) sequences from a fasta or MSA file."""
    record = next(SeqIO.parse(filename, "fasta"))
    return record.description, str(record.seq)

def remove_insertions(sequence: str) -> str:
    """ Removes any insertions into the sequence. Needed to load aligned sequences in an MSA. """
    return sequence.translate(translation)

def read_msa(filename: str) -> List[Tuple[str, str]]:
    """ Reads the sequences from an MSA file, automatically removes insertions."""
    return [(record.description, remove_insertions(str(record.seq))) for record in SeqIO.parse(filename, "fasta")]

In [7]:
def extend(a, b, c, L, A, D):
    """
    input:  3 coords (a,b,c), (L)ength, (A)ngle, and (D)ihedral
    output: 4th coord
    """

    def normalize(x):
        return x / np.linalg.norm(x, ord=2, axis=-1, keepdims=True)

    bc = normalize(b - c)
    n = normalize(np.cross(b - a, bc))
    m = [bc, np.cross(n, bc), n]
    d = [L * np.cos(A), L * np.sin(A) * np.cos(D), -L * np.sin(A) * np.sin(D)]
    return c + sum([m * d for m, d in zip(m, d)])


def contacts_from_pdb(
    structure: bs.AtomArray,
    distance_threshold: float = 8.0,
    chain: Optional[str] = None,
) -> np.ndarray:
    mask = ~structure.hetero
    if chain is not None:
        mask &= structure.chain_id == chain

    N = structure.coord[mask & (structure.atom_name == "N")]
    CA = structure.coord[mask & (structure.atom_name == "CA")]
    C = structure.coord[mask & (structure.atom_name == "C")]

    Cbeta = extend(C, N, CA, 1.522, 1.927, -2.143)
    dist = squareform(pdist(Cbeta))

    contacts = dist < distance_threshold
    contacts = contacts.astype(np.int64)
    contacts[np.isnan(dist)] = -1
    return contacts

In [8]:
# Select sequences from the MSA to maximize the hamming distance
# Alternatively, can use hhfilter
def greedy_select(msa: List[Tuple[str, str]], num_seqs: int, mode: str = "max") -> List[Tuple[str, str]]:
    assert mode in ("max", "min")
    if len(msa) <= num_seqs:
        return msa

    array = np.array([list(seq) for _, seq in msa], dtype=np.bytes_).view(np.uint8)

    optfunc = np.argmax if mode == "max" else np.argmin
    all_indices = np.arange(len(msa))
    indices = [0]
    pairwise_distances = np.zeros((0, len(msa)))
    for _ in range(num_seqs - 1):
        dist = cdist(array[indices[-1:]], array, "hamming")
        pairwise_distances = np.concatenate([pairwise_distances, dist])
        shifted_distance = np.delete(pairwise_distances, indices, axis=1).mean(0)
        shifted_index = optfunc(shifted_distance)
        index = np.delete(all_indices, indices)[shifted_index]
        indices.append(index)
    indices = sorted(indices)
    return [msa[idx] for idx in indices]

In [9]:
def compute_precisions(
    predictions: torch.Tensor,
    targets: torch.Tensor,
    src_lengths: Optional[torch.Tensor] = None,
    minsep: int = 6,
    maxsep: Optional[int] = None,
    override_length: Optional[int] = None,  # for casp
):
    if isinstance(predictions, np.ndarray):
        predictions = torch.from_numpy(predictions)
    if isinstance(targets, np.ndarray):
        targets = torch.from_numpy(targets)
    if predictions.dim() == 2:
        predictions = predictions.unsqueeze(0)
    if targets.dim() == 2:
        targets = targets.unsqueeze(0)
    override_length = (targets[0, 0] >= 0).sum()

    # Check sizes
    if predictions.size() != targets.size():
        raise ValueError(
            f"Size mismatch. Received predictions of size {predictions.size()}, "
            f"targets of size {targets.size()}"
        )
    device = predictions.device

    batch_size, seqlen, _ = predictions.size()
    seqlen_range = torch.arange(seqlen, device=device)

    sep = seqlen_range.unsqueeze(0) - seqlen_range.unsqueeze(1)
    sep = sep.unsqueeze(0)
    valid_mask = sep >= minsep
    valid_mask = valid_mask & (targets >= 0)  # negative targets are invalid

    if maxsep is not None:
        valid_mask &= sep < maxsep

    if src_lengths is not None:
        valid = seqlen_range.unsqueeze(0) < src_lengths.unsqueeze(1)
        valid_mask &= valid.unsqueeze(1) & valid.unsqueeze(2)
    else:
        src_lengths = torch.full([batch_size], seqlen, device=device, dtype=torch.long)

    predictions = predictions.masked_fill(~valid_mask, float("-inf"))

    x_ind, y_ind = np.triu_indices(seqlen, minsep)
    predictions_upper = predictions[:, x_ind, y_ind]
    targets_upper = targets[:, x_ind, y_ind]

    topk = seqlen if override_length is None else max(seqlen, override_length)
    indices = predictions_upper.argsort(dim=-1, descending=True)[:, :topk]
    topk_targets = targets_upper[torch.arange(batch_size).unsqueeze(1), indices]
    if topk_targets.size(1) < topk:
        topk_targets = F.pad(topk_targets, [0, topk - topk_targets.size(1)])

    cumulative_dist = topk_targets.type_as(predictions).cumsum(-1)

    gather_lengths = src_lengths.unsqueeze(1)
    if override_length is not None:
        gather_lengths = override_length * torch.ones_like(
            gather_lengths, device=device
        )

    gather_indices = (
        torch.arange(0.1, 1.1, 0.1, device=device).unsqueeze(0) * gather_lengths
    ).type(torch.long) - 1

    binned_cumulative_dist = cumulative_dist.gather(1, gather_indices)
    binned_precisions = binned_cumulative_dist / (gather_indices + 1).type_as(
        binned_cumulative_dist
    )

    pl5 = binned_precisions[:, 1]
    pl2 = binned_precisions[:, 4]
    pl = binned_precisions[:, 9]
    auc = binned_precisions.mean(-1)

    return {"AUC": auc, "P@L": pl, "P@L2": pl2, "P@L5": pl5}


def evaluate_prediction(
    predictions: torch.Tensor,
    targets: torch.Tensor,
) -> Dict[str, float]:
    if isinstance(targets, np.ndarray):
        targets = torch.from_numpy(targets)
    contact_ranges = [
        ("local", 3, 6),
        ("short", 6, 12),
        ("medium", 12, 24),
        ("long", 24, None),
    ]
    metrics = {}
    targets = targets.to(predictions.device)
    for name, minsep, maxsep in contact_ranges:
        rangemetrics = compute_precisions(
            predictions,
            targets,
            minsep=minsep,
            maxsep=maxsep,
        )
        for key, val in rangemetrics.items():
            metrics[f"{name}_{key}"] = val.item()
    return metrics

In [10]:
"""Adapted from: https://github.com/rmrao/evo/blob/main/evo/visualize.py"""
def plot_contacts_and_predictions(
    predictions: Union[torch.Tensor, np.ndarray],
    contacts: Union[torch.Tensor, np.ndarray],
    ax: Optional[mpl.axes.Axes] = None,
    # artists: Optional[ContactAndPredictionArtists] = None,
    cmap: str = "Blues",
    ms: float = 1,
    title: Union[bool, str, Callable[[float], str]] = True,
    animated: bool = False,
) -> None:

    if isinstance(predictions, torch.Tensor):
        predictions = predictions.detach().cpu().numpy()
    if isinstance(contacts, torch.Tensor):
        contacts = contacts.detach().cpu().numpy()
    if ax is None:
        ax = plt.gca()

    seqlen = contacts.shape[0]
    relative_distance = np.add.outer(-np.arange(seqlen), np.arange(seqlen))
    bottom_mask = relative_distance < 0
    masked_image = np.ma.masked_where(bottom_mask, predictions)
    invalid_mask = np.abs(np.add.outer(np.arange(seqlen), -np.arange(seqlen))) < 6
    predictions = predictions.copy()
    predictions[invalid_mask] = float("-inf")

    topl_val = np.sort(predictions.reshape(-1))[-seqlen]
    pred_contacts = predictions >= topl_val
    true_positives = contacts & pred_contacts & ~bottom_mask
    false_positives = ~contacts & pred_contacts & ~bottom_mask
    other_contacts = contacts & ~pred_contacts & ~bottom_mask

    if isinstance(title, str):
        title_text: Optional[str] = title
    elif title:
        long_range_pl = compute_precisions(predictions, contacts, minsep=24)[
            "P@L"
        ].item()
        if callable(title):
            title_text = title(long_range_pl)
        else:
            title_text = f"Long Range P@L: {100 * long_range_pl:0.1f}"
    else:
        title_text = None

    img = ax.imshow(masked_image, cmap=cmap, animated=animated)
    oc = ax.plot(*np.where(other_contacts), "o", c="grey", ms=ms)[0]
    fn = ax.plot(*np.where(false_positives), "o", c="r", ms=ms)[0]
    tp = ax.plot(*np.where(true_positives), "o", c="b", ms=ms)[0]
    ti = ax.set_title(title_text) if title_text is not None else None
    # artists = ContactAndPredictionArtists(img, oc, fn, tp, ti)

    ax.axis("square")
    ax.set_xlim([0, seqlen])
    ax.set_ylim([0, seqlen])

# TODO: Figure out how to analyze MSATransformer output in a way that makes sense

In [ ]:
#@title Read Data
# This is where the data is actually read in
import os

pdbId0 = '1k0n' #@param {type:"string"}
#@markdown - Specify the 1st PDB ID to compare
pdbId1 = '1rk4' #@param {type:"string"}
#@markdown - Specify the 2nd PDB ID to compare
PDB_IDS = [pdbId0, pdbId1]

structures = {
    name.lower(): get_structure(PDBxFile.read(rcsb.fetch(name, "cif")))[0]
    for name in PDB_IDS
}

contacts = {
    name: contacts_from_pdb(structure, chain="A")
    for name, structure in structures.items()
}

msas = {
        name: read_msa(f"/content/MSATransformer_output/{name}")
    for name in os.listdir("/content/MSATransformer_output/")
}

sequences = {
    name: msa[0] for name, msa in msas.items()
}

### Predict and Visualize via ESM

In [ ]:
#@title ESM-2 pred
esm2, esm2_alphabet = esm.pretrained.esm2_t33_650M_UR50D()
esm2 = esm2.eval().cuda()
esm2_batch_converter = esm2_alphabet.get_batch_converter()
esm2_predictions = {}
esm2_results = []
for name, inputs in sequences.items():
    esm2_batch_labels, esm2_batch_strs, esm2_batch_tokens = esm2_batch_converter([inputs])
    esm2_batch_tokens = esm2_batch_tokens.to(next(esm2.parameters()).device)
    esm2_predictions[name] = esm2.predict_contacts(esm2_batch_tokens)[0].cpu()
    metrics = {"id": name, "model": "ESM-2 (Unsupervised)"}
    metrics.update(evaluate_prediction(esm2_predictions[name], contacts[name]))
    esm2_results.append(metrics)
esm2_results = pd.DataFrame(esm2_results)
display(esm2_results)

In [ ]:
#@title ESM-2 Graph
fig, axes = plt.subplots(figsize=(18, 6), ncols=len(PDB_IDS))
for ax, name in zip(axes, PDB_IDS):
    prediction = esm2_predictions[name]
    target = contacts[name]
    plot_contacts_and_predictions(
        prediction, target, ax=ax, title = lambda prec: f"{name}: Long Range P@L: {100 * prec:0.1f}"
    )
plt.show()

### Predict and Visualize via MSA Transformer

In [19]:
#@title MSA Transformer pred
msa_transformer, msa_transformer_alphabet = esm.pretrained.esm_msa1b_t12_100M_UR50S()
msa_transformer = msa_transformer.eval().cuda()
msa_transformer_batch_converter = msa_transformer_alphabet.get_batch_converter()
msa_transformer_predictions = {}
msa_transformer_results = []
for name, inputs in msas.items():
    inputs = greedy_select(inputs, num_seqs=128) # can change this to pass more/fewer sequences
    msa_transformer_batch_labels, msa_transformer_batch_strs, msa_transformer_batch_tokens = msa_transformer_batch_converter([inputs])
    msa_transformer_batch_tokens = msa_transformer_batch_tokens.to(next(msa_transformer.parameters()).device)
    msa_transformer_predictions[name] = msa_transformer.predict_contacts(msa_transformer_batch_tokens)[0].cpu()
    metrics = {"id": name, "model": "MSA Transformer (Unsupervised)"}
    metrics.update(evaluate_prediction(msa_transformer_predictions[name], contacts[name]))
    msa_transformer_results.append(metrics)
msa_transformer_results = pd.DataFrame(msa_transformer_results)
display(msa_transformer_results)

In [ ]:
#@title MSA Transformer Graph
fig, axes = plt.subplots(figsize=(18, 6), ncols=len(PDB_IDS))
for ax, name in zip(axes, PDB_IDS):
    prediction = msa_transformer_predictions[name]
    target = contacts[name]
    plot_contacts_and_predictions(
        prediction, target, ax=ax, title = lambda prec: f"{name}: Long Range P@L: {100 * prec:0.1f}"
    )
plt.show()